In [173]:
import math
import random as rd
import sys

def algo_Euclide(a,b):
    if(b==0):
        return a
    else:
        return algo_Euclide(b,a%b)
        
def exponentiation_modulaire (a, e, n):
    if e == 0:
        return 1
    elif e%2 == 0:
        return exponentiation_modulaire((a*a)%n, e//2, n)
    else:
        return (a*exponentiation_modulaire((a*a)%n, (e-1)//2, n))%n

def algo_Euclide_etendu(a,b):
    if(b==0):
        return(a,1,0)
    else:
        d,u,v=algo_Euclide_etendu(b, a%b)
        return d,v,u-(a//b)*v
        
def gen_pochon(n):
    tab = []
    somme = 0
    for _ in range(n):
        b = rd.randint(somme + 1, somme + sys.maxsize // n)
        somme += b
        tab.append(b)
    return tab

def gen_W(M):
    W=rd.randint(1, M-1)
    while(math.gcd(W,M)!=1):
        W=rd.randint(1, M-1)
    return W

def gen_permutation(n):
    sigma = list(range(1, n+1))
    rd.shuffle(sigma)
    return sigma

def gen_cle_publique(cle_privee):
    pochon, M, W, sigma = cle_privee
    return [(pochon[sigma[i]-1] * W) % M for i in range(len(pochon))]

## Génération clé privée, problème de pochon et déchiffrement

In [200]:

from random import randint

"""
Génère une clé privée du cryptosystème de Kifli. Une clé privée contient un pochon P,
un entier M tel que la somme du pochon < M, W entier aléatoire entre 1 et M et premier avec ce dernier,
et sigma, représentant une permutation aléatoire des indices du pochon.

Paramètres:
    n : La taille du pochon et de sigma.

Retourne:
    (p, m, w, sigma) : La clé privée sous forme de n-uplet.
"""
def gen_cle_privee(n):
    # cle_privee = (P, M, W, sigma)
    # P = {b1, b2...}
    # M = M tel que somme bi < M
    # W = random entre 1 et M-1, premier avec M
    # sigma = permu aléatoire de P

    # Calcul de P 
    p = gen_pochon(n)

    # Calcul de M
    m = randint(sum(p) + 1, sum(p) + 3)

    # Calcul de W
    w = gen_W(m)

    # Calcul de sigma
    sigma = gen_permutation(n)
    
    return (p, m, w, sigma)

"""
Renvoie une solution au problème de pochon.

Paramètres :
    pochon : Le pochon dont on veut résoudre son problème.
    c : Le nombre à comparer avec les éléments du pochon.

Retour :
    E : La liste des indices du pochon qui résolvent le problème de pochon.
"""
def solve_pochon(pochon, D):
    E = [0] * len(pochon)
    for i in range(len(pochon)):
        if pochon[i] <= D:
            E[i] = 1
            D -= pochon[i]
    return E
"""
Déchiffre un message chiffré par le cryptosystème de Kifli.

Paramètres:
    message_chiffre : Le message chiffré par le cryptosystème de Kifli. Il est
    représenté par une liste, qui sert de découpage du message.
    cle_privee : La clé privée utilisée.

Retourne :
    message_dechiffre : Le message déchiffré, sous forme de liste de listes. Chaque
    sous-liste est un bloc du message.
"""
def dechiffre(message_chiffre, cle_privee):
    # Inverse modulaire de W
    inverse_mod_w = algo_Euclide_etendu(cle_privee[2], cle_privee[1])[1] % cle_privee[1]
    
    # Nous avons considéré que la taille du dernier bloc de message devait être utilisée. Elle est le premier élément du message.
    taille_dernier_bloc = message_chiffre[0]

    message_dechiffre = ""
    for message in message_chiffre[1:]:
        # Déterminer D tel que D = W^-1*message%M, avec W^-1 l'inverse modulaire de W.
        D = inverse_mod_w * message % cle_privee[1]
        print("D=",D)
        print("message=",message)
        # Résolution du problème de pochons
        P = cle_privee[0]
        P.reverse() # On utilise le pochon dans l'ordre décroissant
        E = solve_pochon(P, D)
        print("E = ",E)

        sous_message = ["0"]*len(cle_privee[3])
        for i in range(len(cle_privee[3])):
            sous_message[cle_privee[3][i]-1]=str(E[i]) # Ajout de l'indice de la permutation
        
        message_dechiffre = message_dechiffre.join(sous_message)
        
    # Découpage du dernier bloc, pour retirer les derniers éléments non-désirés, dans le cas où n est différent de la taille du message
    message_dechiffre = message_dechiffre[:len(message_dechiffre)-(len(P)-taille_dernier_bloc)]
    return message_dechiffre

        


# Chiffrement d'un message 

In [168]:
'''
Chiffre un message à partir d'une clé publique
d'un crypto-système de Kifli

Parametres :
    message      (str)   : Message encodé en binaire. ex : '10110'
    cle_publique (int[]) : Tableau d'entier correspondant à la clé publique

Retourne :
    int[] : Une liste de bloc de message crypté
'''
def chiffre(message, cle_publique):
    blocsMessageChiffre = []
    n = len(cle_publique) # n correspond à la taille maximum des 

    # On découpe tout d'abord le message en sous message d'une taille égale ou inférieure à la taille de la clé publique
    # Puis on parcours chacun de ces blocs

    for blocMessage in decoupMessage(message,n) :
        tailleDernierBloc = len(blocMessage)
        sommeC = 0
        for i in range (len(blocMessage)):
            if (blocMessage[i] == '1'):
                # On ajoute à la somme la valeur associé à la position du 1 dans le message qui est la valeur à l'index i du paquet A
                sommeC += cle_publique[i]
        blocsMessageChiffre.append((sommeC))
    return [tailleDernierBloc] + blocsMessageChiffre

'''
Découpe un message en plusieurs chaînes de caractères d'une taille n ou inférieure.
Seulement le dernier bloc peut avoir une taille inférieure.

Parametres :
    message (str) : Message à découper
    n       (int) : Taille 

Retourne :
    str[] : Une liste de blocs du message

'''
def decoupMessage(message,n):
    messageDecoupe = []

    #Découpage jusqu'à que le message initial soit vide
    while len(message) > 0 :
        messageDecoupe.append(message[:n]) # Récupèration d'un bloc de taille n
        message = message[n:] # Chagement du message afin d'enlever le bloc récupéré précedemment
    return messageDecoupe

## Test unitaires

In [169]:
cle_publique_test = [22,16,71,54,56]
n = 5
P = [2,5,11,23,55]
M = 113
W = 27
sigma = (2,5,3,1,4)
cle_priv_test=(P,M,W,sigma)

### Génération de clé publique

In [170]:
print("Clé publique généré à partir de la clé privée : ",gen_cle_publique(cle_priv_test))
assert(gen_cle_publique(cle_priv_test) == cle_publique_test)

Clé publique généré à partir de la clé privée :  [22, 16, 71, 54, 56]


### Chiffrement

In [171]:
messageChiffre = chiffre('10101',cle_publique_test)
print("Test message chiffre n = len(message) : ",messageChiffre)
assert([5,149] == messageChiffre)

messageChiffre = chiffre('10100110',cle_publique_test)
print("Test message chiffre n < len(message) : ",messageChiffre)
assert([3,93,38]  == messageChiffre)

messageChiffre = chiffre('1101',cle_publique_test)
print("Test message chiffre n > len(message) : ",messageChiffre)
assert([4,92] == messageChiffre)

Test message chiffre n = len(message) :  [5, 149]
Test message chiffre n < len(message) :  [3, 93, 38]
Test message chiffre n > len(message) :  [4, 92]


### Dechiffrement

In [217]:
messageDeChiffre = dechiffre([5,149],cle_priv_test)
print("Test message dechiffre n = len(message) : ",messageDeChiffre)
assert("10101" == messageDeChiffre)

messageDeChiffre = dechiffre([1,93,22],cle_priv_test)
print("Test message dechiffre n < len(message) : ",messageDeChiffre)
assert("10100110"== messageDeChiffre)

messageDeChiffre = dechiffre([1,93,22],cle_priv_test)
print("Test message dechiffre n > len(message) : ",messageDeChiffre)
assert("1101" == messageDeChiffre)

D= 39
message= 149
E =  [0, 1, 1, 1, 0]
Test message dechiffre n = len(message) :  10101
D= 16
message= 93
E =  [1, 1, 0, 0, 0]
D= 5
message= 22
E =  [0, 0, 0, 1, 0]
Test message dechiffre n < len(message) :  101001001001001001001


AssertionError: 

## Tests fonctionnels

In [ ]:
def test_chiffrement_dechiffrement(message, n) :
    cle_privee = gen_cle_privee(n)
    cle_publique = gen_cle_publique(cle_privee)
    messageChiffre = chiffre(message, cle_publique)
    print("Pochon=", cle_privee[0])
    print("M=", cle_privee[1])
    print("Message chiffré=",messageChiffre)
    print("Sigma=",cle_privee[3])
    print("clé pub=", cle_publique)
    messageDechiffre = dechiffre(messageChiffre, cle_privee)
    print("Message déchiffré=",messageDechiffre)
    return message == messageDechiffre

test_chiffrement_dechiffrement("10101", 5)

Pochon= [1800790184557121884, 2778276198235750332, 6277356922990549831, 10885081577229533406, 22005569886941366391]
M= 43747074769954321847
Message chiffré= [5, 46560552281089666034]
Sigma= [5, 3, 2, 1, 4]
clé pub= [34032045892420303576, 31516946230063070075, 11361196140261689974, 35202031161057430699, 1167310248407672484]
D= 35668927662406650129
message= 46560552281089666034
E =  [0, 1, 1, 1, 1]
Message déchiffré= [['1', '1', '1', '1', '0']]


False

## D'après vous, le NIST a-t-il eu raison de ne pas retenir le crypto-système de Kifli dans sa liste d'algorithmes post-quantique ? Justifiez votre réponse.


N'ayant pas pu trouver d'informations complémentaire sur le crypto-système de Kifli donc à partir des informations donnés dans le sujet, il nous semble que ce crypto-système mérite une place dans la liste des algorithmes post-quantiques pour deux raisons.

En premier lieu, ce crypto-système semble respecter toutes les qualités que doivent respecter un crypto-système c'est à dire : 
- Confusion : Le message est intelligible lorsqu'il est chiffré.
- Diffusion : Une modification légère du message change le message suffisament pour ne pas déduire des informations.
- Robustesse de la clé : La robustesse de la clé dépend grandement de la capacité à générer aléatoirement une paire clé publique clé privée efficacement, de plus la longueur de la clé doit être assez grande pour qu'elle soit robuste. Cependant si elle est bien généré alors la clé est bien robuste.

En second lieu, il rompt avec le problème de factorisation que propose l'algorithme clé publique, clé privée utilisé en ce moment (RSA) qui est un problème qui sera résolu avec l'apparition des ordinateurs quantiques.
En proposant à la place un problème NP-complet qui n'ai pas d'algorithmes permettant de résoudre tous les cas en compléxité polynomial n.
De plus cet algorithme a peu de chance d'exister dans le futur car il n'est pas certain qu'il puisse exister un tel algorithme.


Cependant, il nous semble pertinent de mentionner qu'en tant que 2ème année de BUT Informatique nous connaissons pas assez la cryptographie et la cryptanalyse nous permettant d'avoir le recul nécessaire pour affirmer que cet algorithme soit viable ou non pour le futur.

